In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *
%nbdev_default_export shape
%nbdev_default_class_level 3

Cells will be exported to geomechy.shape,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
import numpy as np
from scipy.special.orthogonal import p_roots as gauss_scheme
np.set_printoptions(precision=4)

# Shape Functions
> Shape functions routines

## Get Element Type

In [ ]:
%nbdev_export
def getElemetType(elemCoords):
    "Determine the element type"
    
    dict = {
        "numDim_1": {
            "numNodes_2": "Line2", 
            "numNodes_3": "Line3"
            },
        "numDim_2": {
            "numNodes_3": "Tria3",
            "numNodes_4": "Quad4",
            "numNodes_6": "Tria6",
            "numNodes_8": "Quad8",
            "numNodes_9": "Quad9",
        },
        "numDim_3": {
            "numNodes_4":  "Tetra4", 
            "numNodes_8":  "Hexa8",
            "numNodes_10": "Tetra10",
            "numNodes_20": "Hexa20"
            },
    }

    try:
        numNodes = elemCoords.shape[0]
        numDim   = elemCoords.shape[1] if elemCoords.shape[1] else 1
    
        ElemType = dict.get(f"numDim_{numDim}").get(f"numNodes_{numNodes}")

        if ElemType:
            return ElemType
        else:
            raise NotImplementedError(
                f"No {numDim}D element with {numNodes} nodes is available"
            )
    except NotImplementedError as error:
        print(error)
    except IndexError:
        print("No valid coordinates array")
    except AttributeError:
        print("No valid coordinates array")
    except TypeError:
        print("No valid coordinates array")

## Get Gauss Points

### Tria Scheme

In [ ]:
%nbdev_export
def tria_scheme(order):
    
    if order is 1:
        xi     = [[1./3., 1./3.]]
        weight = [[1.]]
    
    elif order is 3:
        r1 = 1./6.
        r2 = 2./3.
        w1 = 1./3.
        
        xi = [[r1,r1],[r2,r1],[r1,r2]]
        weight = [[w1],[w1],[w1]]
    
    elif order is 4:
        r1 = 1./5.
        r2 = 3./5.
        r3 = 1./3.
        
        w1 =  0.52083333
        w2 =  0.52083333
        w3 =  0.52083333
        w4 = -0.56250000

        xi = [[r1,r1],[r2,r1],[r1,r2],[r3,r3]]
        weight = [[w1],[w2],[w3],[w4]]
    
    return xi, weight

### Tetra Scheme

In [ ]:
%nbdev_export
def tetra_scheme(order):
    
    if order is 1:
        xi     = [[1./4., 1./4., 1./4.]]
        weight = [[1.]]
    
    elif order is 4:
        r1 = 0.5854102
        r2 = 0.1381966
        w1 = 1./4.
        
        xi = [[r1,r2,r2],[r2,r1,r2],[r2,r2,r1],[r2,r2,r2]]
        weight = [[w1],[w1],[w1],[w1]]
    
    elif order is 5:
        r1 = 1./4.
        r2 = 1./2.
        r3 = 1./6.
        
        w1 =  9./20.
        w2 = -4./5.

        xi = [[r2,r3,r3],[r3,r2,r3],[r3,r3,r2],[r3,r3,r3],[r1,r1,r1]]
        weight = [[w1],[w1],[w1],[w1],[w2]]
    
    return xi, weight

### Gauss Points and Weights

In [ ]:
%nbdev_export
def getGaussPoints(elemType, reduced=False):

    point = []
    weight = []
 

    if "Line" in elemType:
        stdOrder = 2 if "2" in elemType else 3
        if reduced: stdOrder -= 1
        ip, w = gauss_scheme(stdOrder)
        
        point  = [[ip[i]] for i in range(stdOrder)]
        weight = [[w[i]]  for i in range(stdOrder)]

        
    elif "Tria" in elemType:
        stdOrder = 1 if "3" in elemType else 4
        if stdOrder == 4 and reduced: stdOrder = 3
        
        point, weight = tria_scheme(stdOrder)

        
    elif "Quad" in elemType:
        stdOrder = 2 if "4" in elemType else 3
        if reduced: stdOrder -= 1
        ip, w = gauss_scheme(stdOrder) 
        
        point  = [[ip[j], ip[i]] for i in range(stdOrder) for j in range(stdOrder)]
        weight = [[w[j]*w[i]]    for i in range(stdOrder) for j in range(stdOrder)]

        
    elif "Tetra" in elemType:
        stdOrder = 1 if "4" in elemType else 5
        if stdOrder == 5 and reduced: stdOrder = 4
        
        point, weight = tetra_scheme(stdOrder)
    
    
    elif "Hexa" in elemType:
        stdOrder = 2 if "8" in elemType else 3
        if reduced: stdOrder -= 1
        ip, w = gauss_scheme(stdOrder)
        
        point = [[ip[k], ip[j], ip[i]] for i in range(stdOrder) for j in range(stdOrder) for k in range(stdOrder)]
        weight = [[w[k]*w[j]*w[i]]     for i in range(stdOrder) for j in range(stdOrder) for k in range(stdOrder)]

        
    return np.array(point), np.array(weight)

## Get Shape Functions 1D

### Line2

In [ ]:
%nbdev_export
def getShapeLine2(gaussPoint):

    # Check the dimension of physical space
    if gaussPoint.shape[0] != 1:
        raise NotImplementedError("1D only")
        
    ############################################################################################################
    # gauss points coords
    xi = gaussPoint[0]
    
    ############################################################################################################
    # Tuple with xi_a combinatory
    xi_comb = [-1,1]
    
    ############################################################################################################
    # Calculate shape functions
    N = np.array([0.5*(1+sign*xi) for sign in xi_comb])
    
    ############################################################################################################
    # Calculate derivatives of shape functions-> xi
    dN  = np.array([0.5*sign for sign in xi_comb])

    return N, dN

### Line3

In [ ]:
%nbdev_export
def getShapeLine3(gaussPoint):

    # Check the dimension of physical space
    if gaussPoint.shape[0] != 1:
        raise NotImplementedError("1D only")
    
    ############################################################################################################
    # gauss points coords
    xi = gaussPoint[0]
    
    ############################################################################################################
    # Tuple with xi_a combinatory
    xi_comb = [-1,1]
    
    ############################################################################################################
    # Calculate shape functions
    N_lateral = np.array([0.5*item*xi*(1+item*xi) for item in xi_comb])
    N_middle  = np.array([(1+xi)*(1-xi)])
    
    N = np.hstack((N_lateral[0], N_middle, N_lateral[1]))
    
    ############################################################################################################
    # Calculate derivatives of shape functions -> xi
    dN_lateral = np.array([0.5*item*(1+2.*item*xi) for item in xi_comb])
    dN_middle  = np.array([-2.*xi])
    
    dN = np.hstack((dN_lateral[0], dN_middle, dN_lateral[1]))

    return N, dN

## Get Shape Functions 2D

### Tria3

In [ ]:
%nbdev_export
def getShapeTria3(gaussPoint):

    # Check the dimension of physical space
    if gaussPoint.shape[0] != 2:
        raise NotImplementedError("2D only")
    
    ############################################################################################################
    # gauss points coords
    L1 = gaussPoint[0]
    L2 = gaussPoint[1]
    L3 = 1-L1-L2
    
    ############################################################################################################
    # Calculate shape functions
    N = np.array([L1, L2, L3])
    
    ############################################################################################################
    # Calculate derivatives of shape functions-> xi
    dN_dxi  = np.array([1., 0., -1.])
    
    # Calculate derivatives of shape functions-> eta
    dN_deta = np.array([0., 1., -1.])
    
    # Calculate derivatives of shape functions
    dN = np.vstack((dN_dxi, dN_deta))

    return N, dN

### Tria6

In [ ]:
%nbdev_export
def getShapeTria6(gaussPoint):

    # Check the dimension of physical space
    if gaussPoint.shape[0] != 2:
        raise NotImplementedError("2D only")
    
    ############################################################################################################
    # gauss points coords
    L1 = gaussPoint[0]
    L2 = gaussPoint[1]
    L3 = 1-gaussPoint[0]-gaussPoint[1]
    
    ############################################################################################################
    # Calculate shape functions
    N = np.array([L1*(2.*L1-1.), L2*(2.*L2-1.), L3*(2.*L3-1.), 4*L1*L2, 4*L2*L3, 4*L1*L3])

    ############################################################################################################
    # Calculate derivatives of shape functions-> xi
    dN_dxi  = np.array([4.*L1-1, 0., -4.*L3+1, 4.*L2, -4.*L2, 4.*(L3-L1)])
    
    # Calculate derivatives of shape functions-> eta
    dN_deta = np.array([0., 4.*L2-1, -4.*L3+1, 4.*L1, 4.*(L3-L2), -4.*L1])
    
    # Calculate derivatives of shape functions
    dN = np.vstack((dN_dxi, dN_deta))

    return N, dN

### Quad4

In [ ]:
%nbdev_export
def getShapeQuad4(gaussPoint):

    # Check the dimension of physical space
    if gaussPoint.shape[0] != 2:
        raise NotImplementedError("2D only")
        
    ############################################################################################################
    # gauss points coords
    xi  = gaussPoint[0]
    eta = gaussPoint[1]
    
    ############################################################################################################
    # Tuple with xi_a and eta_a combinatory
    xi_eta_comb = [(-1,-1),(1,-1),(1,1),(-1,1)]
    
    ############################################################################################################
    # Calculate shape functions
    N = np.array([0.25*(1.0+sign[0]*xi)*(1.0+sign[1]*eta) for sign in xi_eta_comb])
    
    ############################################################################################################
    # Calculate derivatives of shape functions-> xi
    dN_dxi  = np.array([0.25*sign[0]*(1+sign[1]*eta) for sign in xi_eta_comb])
    
    # Calculate derivatives of shape functions-> eta
    dN_deta = np.array([0.25*sign[1]*(1+sign[0]*xi) for sign in xi_eta_comb])
    
    # Calculate derivatives of shape functions
    dN = np.vstack((dN_dxi, dN_deta))

    return N, dN

### Quad8

In [ ]:
%nbdev_export
def getShapeQuad8(gaussPoint):

    # Check the dimension of physical space
    if gaussPoint.shape[0] != 2:
        raise NotImplementedError("2D only")
        
        
    ############################################################################################################
    # gauss points coords
    xi  = gaussPoint[0]
    eta = gaussPoint[1]
    
    ############################################################################################################
    # Tuple with xi_a and eta_a combinatory
    xi_eta_comb = [(-1,-1),(1,-1),(1,1),(-1,1)]

    ############################################################################################################
    # Calculate shape functions
    # Nodes -> 1,2,3,4
    N_lateral = np.array([0.25*(1+sign[0]*xi)*(1+sign[1]*eta)*(sign[0]*xi+sign[1]*eta-1) for sign in xi_eta_comb])
    
    # Nodes -> 5,7
    N_middle_xi  = np.array([0.5*(1-xi**2)*(1+sign*eta) for sign in [-1,1]])
    
    # Nodes -> 6,8
    N_middle_eta = np.array([0.5*(1-eta**2)*(1+sign*xi) for sign in [1,-1]])
    
    # Nodes -> ALL
    N = np.hstack((N_lateral, N_middle_xi[0], N_middle_eta[0], N_middle_xi[1], N_middle_eta[1]))
    
    
    ############################################################################################################
    # Calculate derivatives of shape functions -> xi
    # Nodes -> 1,2,3,4
    dN_dxi_lateral = np.array([0.25*sign[0]*(1+sign[1]*eta)*(2*sign[0]*xi+sign[1]*eta) for sign in xi_eta_comb])
    
    # Nodes -> 5,7
    dN_dxi_middle_xi  = np.array([0.5*(1+sign*eta)*(-2.*xi) for sign in [-1,1]])
    
    # Nodes -> 6,8
    dN_dxi_middle_eta = np.array([0.5*(1-eta**2)*sign for sign in [1,-1]])
    
    # Nodes -> ALL
    dN_dxi = np.hstack((dN_dxi_lateral, dN_dxi_middle_xi[0], dN_dxi_middle_eta[0], dN_dxi_middle_xi[1], dN_dxi_middle_eta[1]))
    
    
    ############################################################################################################
    # Calculate derivatives of shape functions -> eta
    # Nodes -> 1,2,3,4
    dN_deta_lateral = np.array([0.25*sign[1]*(1+sign[0]*xi)*(2*sign[1]*eta+sign[0]*xi) for sign in xi_eta_comb])
    
    # Nodes -> 5,7
    dN_deta_middle_xi  = np.array([0.5*(1-xi**2)*sign for sign in [-1,1]])
    
    # Nodes -> 6,8
    dN_deta_middle_eta = np.array([0.5*(1+sign*xi)*(-2.*eta) for sign in [1,-1]])
    
    # Nodes -> ALL
    dN_deta = np.hstack((dN_deta_lateral, dN_deta_middle_xi[0], dN_deta_middle_eta[0], 
                         dN_deta_middle_xi[1], dN_deta_middle_eta[1]))
    
    
    ############################################################################################################
    # Calculate derivatives of shape functions
    dN = np.vstack((dN_dxi, dN_deta))

    return N, dN

### Quad9

In [ ]:
%nbdev_export
def getShapeQuad9(gaussPoint):

    # Check the dimension of physical space
    if gaussPoint.shape[0] != 2:
        raise NotImplementedError("2D only")
        
        
    ############################################################################################################
    # gauss points coords
    xi  = gaussPoint[0]
    eta = gaussPoint[1]
    
    ############################################################################################################
    # Tuple with xi_a and eta_a combinatory
    xi_eta_comb = [(-1,-1),(1,-1),(1,1),(-1,1)]

    ############################################################################################################
    # Calculate shape functions
    # Nodes -> 1,2,3,4
    N_lateral = np.array([0.25*xi*eta*(xi+sign[0])*(eta+sign[1]) for sign in xi_eta_comb])
    
    # Nodes -> 5,7
    N_middle_xi  = np.array([0.5*eta*(1-xi**2)*(eta+sign) for sign in [-1,1]])
    
    # Nodes -> 6,8
    N_middle_eta = np.array([0.5*xi*(1-eta**2)*(xi+sign) for sign in [1,-1]])
    
    # Nodes -> 9
    N_center_eta = np.array((1-xi**2)*(1-eta**2))
    
    # Nodes -> ALL
    N = np.hstack((N_lateral, N_middle_xi[0], N_middle_eta[0], N_middle_xi[1], N_middle_eta[1], N_center_eta))
    
    
    ############################################################################################################
    # Calculate derivatives of shape functions -> xi
    # Nodes -> 1,2,3,4
    dN_dxi_lateral = np.array([0.25*eta*(eta+sign[1])*(2*xi+sign[0]) for sign in xi_eta_comb])
    
    # Nodes -> 5,7
    dN_dxi_middle_xi  = np.array([0.5*eta*(eta+sign)*(-2.*xi) for sign in [-1,1]])
    
    # Nodes -> 6,8
    dN_dxi_middle_eta = np.array([0.5*(1-eta**2)*(2*xi+sign) for sign in [1,-1]])
    
    # Nodes -> 9
    dN_dxi_center_eta = np.array((1-eta**2)*(-2*xi))
    
    # Nodes -> ALL
    dN_dxi = np.hstack((dN_dxi_lateral, dN_dxi_middle_xi[0], dN_dxi_middle_eta[0], 
                        dN_dxi_middle_xi[1], dN_dxi_middle_eta[1], dN_dxi_center_eta))
    
    
    ############################################################################################################
    # Calculate derivatives of shape functions -> eta
    # Nodes -> 1,2,3,4
    dN_deta_lateral = np.array([0.25*xi*(xi+sign[0])*(2*eta+sign[1]) for sign in xi_eta_comb])
    
    # Nodes -> 5,7
    dN_deta_middle_xi  = np.array([0.5*(1-xi**2)*(2*eta+sign) for sign in [-1,1]])
    
    # Nodes -> 6,8
    dN_deta_middle_eta = np.array([0.5*xi*(xi+sign)*(-2.*eta) for sign in [1,-1]])
    
    # Nodes -> 9
    dN_deta_center_eta = np.array((1-xi**2)*(-2*eta))
    
    # Nodes -> ALL
    dN_deta = np.hstack((dN_deta_lateral, dN_deta_middle_xi[0], dN_deta_middle_eta[0], 
                         dN_deta_middle_xi[1], dN_deta_middle_eta[1], dN_deta_center_eta))
    
    
    ############################################################################################################
    # Calculate derivatives of shape functions
    dN = np.vstack((dN_dxi, dN_deta))

    return N, dN

## Get Shape Functions 3D

### Tetra4

In [ ]:
%nbdev_export
def getShapeTetra4(gaussPoint):

    # Check the dimension of physical space
    if gaussPoint.shape[0] != 3:
        raise NotImplementedError("3D only")
    
    
    ############################################################################################################
    # gauss points coords
    L1 = gaussPoint[0]
    L2 = gaussPoint[1]
    L3 = gaussPoint[2]
    L4 = 1-L1-L2-L3
    
    ############################################################################################################
    # Calculate shape functions
    N = np.array([L1, L2, L3, L4])
    
    ############################################################################################################
    # Calculate derivatives of shape functions-> xi
    dN_dxi  = np.array([1., 0., 0., -1])
    
    # Calculate derivatives of shape functions-> eta
    dN_deta = np.array([0., 1., 0., -1.])
    
    # Calculate derivatives of shape functions-> dseta
    dN_ddseta = np.array([0., 0., 1., -1.])
    
    # Calculate derivatives of shape functions
    dN = np.vstack((dN_dxi, dN_deta, dN_ddseta))

    return N, dN

### Tetra10

In [ ]:
%nbdev_export
def getShapeTetra10(gaussPoint):

    # Check the dimension of physical space
    if gaussPoint.shape[0] != 3:
        raise NotImplementedError("3D only")
    
    
    ############################################################################################################
    # gauss points coords
    L1 = gaussPoint[0]
    L2 = gaussPoint[1]
    L3 = gaussPoint[2]
    L4 = 1-L1-L2-L3
    
    ############################################################################################################
    # Calculate shape functions
    N = np.array([L1*(2*L1-1), L2*(2*L2-1), L3*(2*L3-1), L4*(2*L4-1), 
                  4*L1*L2, 4*L1*L3, 4*L2*L3, 4*L1*L4, 4*L2*L4, 4*L3*L4])
    
    ############################################################################################################
    # Calculate derivatives of shape functions-> xi
    dN_dxi  = np.array([4*L1-1, 0., 0., -4*L4+1,
                       4*L2, 4*L3, 0.,
                       4*(L4-L1), -4*L2, -4*L3])
    
    # Calculate derivatives of shape functions-> eta
    dN_deta = np.array([0., 4*L2-1, 0., -4*L4+1.,
                       4*L1, 0., 4*L3,
                       -4*L1, 4*(L4-L2), -4*L3])
    
    # Calculate derivatives of shape functions-> dseta
    dN_ddseta = np.array([0., 0., 4*L3-1, -4*L4+1,
                         0., 4*L1, 4*L2,
                         -4*L1, -4*L2, 4*(L4-L3)])
    
    # Calculate derivatives of shape functions
    dN = np.vstack((dN_dxi, dN_deta, dN_ddseta))

    return N, dN

### Hexa8

In [ ]:
%nbdev_export
def getShapeHexa8(gaussPoint):

    # Check the dimension of physical space
    if gaussPoint.shape[0] != 3:
        raise NotImplementedError("3D only")
    
    
    ############################################################################################################
    # gauss points coords
    xi    = gaussPoint[0]
    eta   = gaussPoint[1]
    dseta = gaussPoint[2]
    
    ############################################################################################################
    # Tuple with xi_a and eta_a combinatory
    xi_eta_dseta_comb = [(-1,-1,-1),(1,-1,-1),(1,1,-1),(-1,1,-1),
                         (-1,-1, 1),(1,-1, 1),(1,1, 1),(-1,1, 1)]
    
    ############################################################################################################
    # Calculate shape functions
    N = np.array([(1/8)*(1+sign[0]*xi)*(1+sign[1]*eta)*(1+sign[2]*dseta) for sign in xi_eta_dseta_comb])
    
    ############################################################################################################
    # Calculate derivatives of shape functions-> xi
    dN_dxi  = np.array([(1/8)*sign[0]*(1+sign[1]*eta)*(1+sign[2]*dseta) for sign in xi_eta_dseta_comb])
    
    # Calculate derivatives of shape functions-> eta
    dN_deta = np.array([(1/8)*sign[1]*(1+sign[0]*xi)*(1+sign[2]*dseta) for sign in xi_eta_dseta_comb])
    
    # Calculate derivatives of shape functions-> dseta
    dN_ddseta = np.array([(1/8)*sign[2]*(1+sign[0]*xi)*(1+sign[1]*eta) for sign in xi_eta_dseta_comb])
    
    # Calculate derivatives of shape functions
    dN = np.vstack((dN_dxi, dN_deta, dN_ddseta))

    return N, dN

### Hexa20

In [ ]:
%nbdev_export
def getShapeHexa20(gaussPoint):

    # Check the dimension of physical space
    if gaussPoint.shape[0] != 3:
        raise NotImplementedError("3D only")
    
    
    ############################################################################################################
    # gauss points coords
    xi    = gaussPoint[0]
    eta   = gaussPoint[1]
    dseta = gaussPoint[2]
    
    ############################################################################################################
    # Tuple with xi_a, eta_a and dseta_a combinatory
    
    # Nodes -> 1,2,3,4,5,6,7,8
    xi_eta_dseta_1_8 = [(-1,-1,-1),(1,-1,-1),(1,1,-1),(-1,1,-1),
                         (-1,-1, 1),(1,-1, 1),(1,1, 1),(-1,1, 1)]
    
    # Nodes -> 9,11,13,15
    xi_eta_dseta_9_11_13_15  = [(0,-1,-1),(0,1,-1), (0,-1,1),(0,1,1)]
    
    # Nodes -> 10,12,14,16
    xi_eta_dseta_10_12_14_16 = [(1,0,-1), (-1,0,-1),(1,0,1), (-1,0,1)]
    
    # Nodes -> 17,18,19,20
    xi_eta_dseta_17_18_19_20 = [(-1,-1,0), (1,-1,0),(1,1,0), (-1,1,0)]
    
    
    ############################################################################################################
    # Calculate shape functions
    
    # Nodes -> 1,2,3,4,5,6,7,8
    N1 = np.array([(1/8)*(1+sign[0]*xi)*(1+sign[1]*eta)*(1+sign[2]*dseta)
                  *(sign[0]*xi+sign[1]*eta+sign[2]*dseta-2) for sign in xi_eta_dseta_1_8])
    
    # Nodes -> 9,11,13,15
    N2  = np.array([(1/4)*(1-xi**2)*(1+sign[1]*eta)*(1+sign[2]*dseta) for sign in xi_eta_dseta_9_11_13_15])
    
    # Nodes -> 10,12,14,16
    N3 = np.array([(1/4)*(1+sign[0]*xi)*(1-eta**2)*(1+sign[2]*dseta) for sign in xi_eta_dseta_10_12_14_16])
    
    # Nodes -> 17,18,19,20
    N4 = np.array([(1/4)*(1+sign[0]*xi)*(1+sign[1]*eta)*(1-dseta**2) for sign in xi_eta_dseta_17_18_19_20])
    
    # Nodes -> ALL
    N = np.hstack((N1, N2[0], N3[0], N2[1], N3[1], N2[2], N3[2], N2[3], N3[3], N4))
    
    
    ############################################################################################################
    # Calculate derivatives of shape functions-> xi
    
    # Nodes -> 1,2,3,4,5,6,7,8
    dN_dxi_1  = np.array([(1/8)*sign[0]*(1+sign[1]*eta)*(1+sign[2]*dseta)*
                          (2*sign[0]*xi+sign[1]*eta+sign[2]*dseta-1) for sign in xi_eta_dseta_1_8])
    
    # Nodes -> 9,11,13,15
    dN_dxi_2  = np.array([(1/4)*(-2.*xi)*(1+sign[1]*eta)*(1+sign[2]*dseta) for sign in xi_eta_dseta_9_11_13_15])
    
    # Nodes -> 10,12,14,16
    dN_dxi_3  = np.array([(1/4)*sign[0]*(1-eta**2)*(1+sign[2]*dseta) for sign in xi_eta_dseta_10_12_14_16])
    
    # Nodes -> 17,18,19,20
    dN_dxi_4  = np.array([(1/4)*sign[0]*(1-dseta**2)*(1+sign[1]*eta) for sign in xi_eta_dseta_17_18_19_20])
    
    # Nodes -> ALL
    dN_dxi = np.hstack((dN_dxi_1, dN_dxi_2[0], dN_dxi_3[0], dN_dxi_2[1], dN_dxi_3[1],
                        dN_dxi_2[2], dN_dxi_3[2], dN_dxi_2[3], dN_dxi_3[3], dN_dxi_4))
    
    
    ############################################################################################################
    # Calculate derivatives of shape functions-> eta
    
    # Nodes -> 1,2,3,4,5,6,7,8
    dN_deta_1  = np.array([(1/8)*sign[1]*(1+sign[0]*xi)*(1+sign[2]*dseta)*
                          (2*sign[1]*eta+sign[0]*xi+sign[2]*dseta-1) for sign in xi_eta_dseta_1_8])
    
    # Nodes -> 9,11,13,15
    dN_deta_2  = np.array([(1/4)*sign[1]*(1-xi**2)*(1+sign[2]*dseta) for sign in xi_eta_dseta_9_11_13_15])
    
    # Nodes -> 10,12,14,16
    dN_deta_3  = np.array([(1/4)*(-2.*eta)*(1+sign[0]*xi)*(1+sign[2]*dseta) for sign in xi_eta_dseta_10_12_14_16])
    
    # Nodes -> 17,18,19,20
    dN_deta_4  = np.array([(1/4)*sign[1]*(1-dseta**2)*(1+sign[0]*xi) for sign in xi_eta_dseta_17_18_19_20])
    
    # Nodes -> ALL
    dN_deta = np.hstack((dN_deta_1, dN_deta_2[0], dN_deta_3[0], dN_deta_2[1], dN_deta_3[1],
                        dN_deta_2[2], dN_deta_3[2], dN_deta_2[3], dN_deta_3[3], dN_deta_4))
    
    
    ############################################################################################################
    # Calculate derivatives of shape functions-> dseta
    
    # Nodes -> 1,2,3,4,5,6,7,8
    dN_ddseta_1  = np.array([(1/8)*sign[2]*(1+sign[0]*xi)*(1+sign[1]*eta)*
                          (2*sign[2]*dseta+sign[0]*xi+sign[1]*eta-1) for sign in xi_eta_dseta_1_8])
    
    # Nodes -> 9,11,13,15
    dN_ddseta_2  = np.array([(1/4)*sign[2]*(1-xi**2)*(1+sign[1]*eta) for sign in xi_eta_dseta_9_11_13_15])
    
    # Nodes -> 10,12,14,16
    dN_ddseta_3  = np.array([(1/4)*sign[2]*(1-eta**2)*(1+sign[0]*xi) for sign in xi_eta_dseta_10_12_14_16])
    
    # Nodes -> 17,18,19,20
    dN_ddseta_4  = np.array([(1/4)*(-2.*dseta)*(1+sign[0]*xi)*(1+sign[1]*eta) for sign in xi_eta_dseta_17_18_19_20])
    
    # Nodes -> ALL
    dN_ddseta = np.hstack((dN_ddseta_1, dN_ddseta_2[0], dN_ddseta_3[0], dN_ddseta_2[1], dN_ddseta_3[1],
                        dN_ddseta_2[2], dN_ddseta_3[2], dN_ddseta_2[3], dN_ddseta_3[3], dN_ddseta_4))
    
    
    # Calculate derivatives of shape functions
    dN = np.vstack((dN_dxi, dN_deta, dN_ddseta))

    return N, dN

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_physics.ipynb.
Converted 02_materials.ipynb.
Converted 03_shape.ipynb.
Converted 04_io.ipynb.
Converted 08_Modules.ipynb.
Converted 09_base.ipynb.
Converted 10_utils.ipynb.
Converted index.ipynb.
